## Image-to-video stable-video-diffusion-img2vid

Since generating videos is more memory intensive we can use the `decode_chunk_size` argument to control how many frames are decoded at once. This will reduce the memory usage. It's recommended to tweak this value based on your GPU memory. Setting `decode_chunk_size=1` will decode one frame at a time and will use the least amount of memory but the video might have some flickering.

In [1]:
import torch

from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid", torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()  # Use this if you don't have a GPU
# pipe.to("cuda")
# pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

pipe.unet.enable_forward_chunking()  # disable if enough memory as this slows down significantly

# load image as PIL image
image = load_image("./output/text2img/4.0-me-as-jedi.png")
# image = Image.open("./outputs/text2img/4.0-me-as-jedi.png")
# print image resolution
print(image.size)
# resize image to 576x576
image_576 = image.resize((576, 576))
# image = image.resize((1024, 576))
print(image_576.size)

generator = torch.manual_seed(42)
# with torch.inference_mode():
frames = pipe(image_576, decode_chunk_size=1, generator=generator, num_frames=25).frames[0]

# save sample video to outputs/img2vid folder
export_to_video(frames, "output/img2vid/svd-measjedi2.mp4", fps=16)

Couldn't connect to the Hub: 504 Server Error: Gateway Time-out for url: https://huggingface.co/api/models/stabilityai/stable-video-diffusion-img2vid.
Will try to load from local cache.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

(1024, 1024)
(576, 576)


  0%|          | 0/25 [00:00<?, ?it/s]

'outputs/img2vid/4.0-me-as-jedi2.mp4'